In [21]:
import os
import requests
import argparse
from tqdm import tqdm
import sys
import logging

src_dir = "../../src"
sys.path.insert(0, src_dir)

import pump  # noqa: E402
import dspace  # noqa: E402
from project_settings import settings  # noqa: E402
from dotenv import load_dotenv


In [22]:
env_path = os.path.join(src_dir, ".env-zcu")
url = "https://naos-be.zcu.cz/server/api"
load_dotenv(dotenv_path=env_path)

True

In [23]:
# print(os.environ["DSPACE_USER"])
url = url.rstrip("/")
dspace_be = dspace.rest(endpoint=url, user=os.environ["DSPACE_USER"], password=os.environ["DSPACE_PASSWORD"], auth=True)

2025-05-28 18:10:32,621 - Initialise connection to DSpace REST backend [https://naos-be.zcu.cz/server/api]
2025-05-28 18:10:32,881 - Authenticated successfully as dataquest@admin.edu
2025-05-28 18:10:32,881 - DSpace REST backend is available at [https://naos-be.zcu.cz/server/api]


In [24]:
coms = dspace_be.client.get_communities(size=100, top=True)

print(f"Found {len(coms)} coms")
# print(coms)
print([c.name for c in coms])

Found 5 coms
['Dataquest Testing Private Community', 'Otevřené zdroje / Open resources', 'Publikace ZČU / UWB Publications', 'Publikační činnost / Publications', 'Vysokoškolské kvalifikační práce / Theses']


In [25]:
print(coms[0].as_dict())
print(coms[0].links)

{'uuid': '8dfebefa-f044-4da7-a6cd-afe552a975ae', 'name': 'Dataquest Testing Private Community', 'handle': '11025/57425', 'metadata': {'dc.identifier.uri': [{'value': 'http://hdl.handle.net/11025/57425', 'language': None, 'authority': None, 'confidence': -1, 'place': 0}], 'dc.title': [{'value': 'Dataquest Testing Private Community', 'language': None, 'authority': None, 'confidence': -1, 'place': 0}]}, 'lastModified': None, 'type': 'community'}
{'collections': {'href': 'https://naos-be.zcu.cz/server/api/core/communities/8dfebefa-f044-4da7-a6cd-afe552a975ae/collections'}, 'logo': {'href': 'https://naos-be.zcu.cz/server/api/core/communities/8dfebefa-f044-4da7-a6cd-afe552a975ae/logo'}, 'subcommunities': {'href': 'https://naos-be.zcu.cz/server/api/core/communities/8dfebefa-f044-4da7-a6cd-afe552a975ae/subcommunities'}, 'parentCommunity': {'href': 'https://naos-be.zcu.cz/server/api/core/communities/8dfebefa-f044-4da7-a6cd-afe552a975ae/parentCommunity'}, 'adminGroup': {'href': 'https://naos-be.

In [27]:
import tqdm

def _get_all(ftor, **kwargs):
    arr = []
    for page in range(0, sys.maxsize):
        carr = ftor(page=page, size=100, **kwargs)
        if len(carr or []) == 0:
            break
        arr += carr
    return arr

def com2dict(com):
    d = com.as_dict()
    d["_links"] = com.links
    return d
    
def _get_subc_from_links(com):
    d_subc = com.links.get("subcommunities", {})
    if "href" not in d_subc:
        return []
    r_json = dspace_be.client.fetch_resource(d_subc["href"])
    arr = []
    if '_embedded' in r_json:
        arr += r_json['_embedded'].get('subcommunities', [])
    elif 'uuid' in r_json:
        arr = [r_json]
    return arr
    
def _rec_make_com_dict(com_js_arr, top: bool = False):
    com_arr = [dspace_be.models.Community(x) for x in com_js_arr]
    d = {}
    loop = com_arr if not top else tqdm.tqdm(com_arr)
    for com in loop:
        subc_arr = _get_subc_from_links(com)
        cols = [x.as_dict() for x in _get_all(
            dspace_be.client.get_collections, community=com)]
        d[com.uuid] = {
            "self": com2dict(com),
            "cols": cols,
            "subcoms": _rec_make_com_dict(subc_arr)
        }
    return d

top_coms_arr = [com2dict(x) for x in coms]
coms_d = _rec_make_com_dict(top_coms_arr, top=True)

# for c in coms:
#     print(40 * "=")
#     print(c.name)
#     subc_arr = _get_subc_from_links(c)
#     print(subc_arr)
#     cols = dspace_be.client.get_collections(community=c)
#     for c in cols:
#         print(f"\t{c.name}")

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:42<00:00, 20.54s/it]


In [28]:
def _print_com(js_com, arr, stats, level=0):
    for js_col in js_com['cols']:
        col =  dspace_be.models.Collection(js_col)
        arr.append(f"{' ' * (level + 4)}Collection: {col.name}")
        stats["cols"] += 1
    for js_subcom in js_com['subcoms'].values():
        subcom = dspace_be.models.Community(js_subcom['self'])
        arr.append(f"{' ' * (level + 4)}SUB-Community: {subcom.name}")
        stats["subcoms"] += 1
        _print_com(js_subcom, arr, stats, level + 4)

arr = []
stats = {
    "coms": len(coms_d),
    "cols": 0,
    "subcoms": 0
}
for js_com in coms_d.values():
    com = dspace_be.models.Community(js_com['self'])
    arr.append(f"Community: {com.name}")
    _print_com(js_com, arr, stats)
arr.append("====")
arr.append(f"Stats: {stats}")
for l in arr:
    print(l)

Community: Dataquest Testing Private Community
    Collection: Dataquest Testing Private Collection
Community: Otevřené zdroje / Open resources
    SUB-Community: Materiály pro e-learning / Materials for e-learning
        Collection: Příprava materiálů / Material preparation
        Collection: Přírodní vědy / Natural science
    SUB-Community: Materiály pro popularizaci vědy / Materials to popularization of science
        Collection: Popularizace vědy / Popularization of science
        Collection: Přírodní vědy / Natural science
        Collection: Rozhovory / Interviews
    SUB-Community: Materiály pro střední školy / Materials for secondary schools
        Collection: Bádáme v kroužku biologie na SŠ / We research in biology circle at secondary school
        Collection: Bádáme v kroužku chemie na SŠ / We research in chemistry circle at secondary school
        Collection: Bádáme v kroužku ekologie na SŠ / We research in ecology circle at secondary school
        Collection: Bádám

In [29]:
#url = f'{dspace_be.client.API_ENDPOINT}/eperson/groups/76249e48-e378-4aee-a9cf-950e39bf292a/epersons'
url = f'{dspace_be.client.API_ENDPOINT}/authz/authorizations/search/object?uri=https://naos-be.zcu.cz/server/api/core/collections/4fb201c4-aa9a-466c-8abc-cfcb1f57e85c'
#url = f'{dspace_be.client.API_ENDPOINT}/authz/resourcepolicies/search/group?uuid=76249e48-e378-4aee-a9cf-950e39bf292a'

print(url)
resp = dspace_be.client.api_get(url)
print(resp)
# print(resp.text)
print(resp.json())


https://naos-be.zcu.cz/server/api/authz/authorizations/search/object?uri=https://naos-be.zcu.cz/server/api/core/collections/4fb201c4-aa9a-466c-8abc-cfcb1f57e85c
<Response [200]>
{'_embedded': {'authorizations': [{'id': '35febc81-623f-4f8a-bb72-5cd43ee94ff2_administratorOf_core.collection_4fb201c4-aa9a-466c-8abc-cfcb1f57e85c', 'type': 'authorization', '_links': {'eperson': {'href': 'https://naos-be.zcu.cz/server/api/authz/authorizations/35febc81-623f-4f8a-bb72-5cd43ee94ff2_administratorOf_core.collection_4fb201c4-aa9a-466c-8abc-cfcb1f57e85c/eperson'}, 'feature': {'href': 'https://naos-be.zcu.cz/server/api/authz/authorizations/35febc81-623f-4f8a-bb72-5cd43ee94ff2_administratorOf_core.collection_4fb201c4-aa9a-466c-8abc-cfcb1f57e85c/feature'}, 'object': {'href': 'https://naos-be.zcu.cz/server/api/authz/authorizations/35febc81-623f-4f8a-bb72-5cd43ee94ff2_administratorOf_core.collection_4fb201c4-aa9a-466c-8abc-cfcb1f57e85c/object'}, 'self': {'href': 'https://naos-be.zcu.cz/server/api/authz/a